#Import Python Packages

In [1]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/metatlas/metatlas/helpers/' )
import dill2plots as dp
%matplotlib inline

('Metatlas live in ', '/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/metatlas')
you're running on 6e08d5351d88 at 172.17.0.8 


:0: FutureWarning: IPython widgets are experimental and may change in the future.


#Make blank file-info sheet

Get files from database

In [10]:
files = dp.get_metatlas_files(experiment = '%hedlund%',name = '%hilic%',most_recent = True)

Save the files to an empty fileinfo sheet

In [9]:
dp.make_empty_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/hedlund_empty_fileinfo.tab',files)

#Create metatlas groups from filled in file-info sheet

In [18]:
dp = reload(dp)
g = dp.make_groups_from_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/FileInfoSheet_BHedlund_AllFiles_20160413 (1).txt',store=False)

#Create a new Atlas in metatlas from a spreadsheet

In [28]:
dp = reload(dp)
names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/katherine_master_pos.txt',
                                       'master_atlas_pos_hilic_20160421',
                                       filetype='tab',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=False)

#Select groups of files to operate on

In [2]:
groups = dp.select_groups_for_analysis(name = '%hedlund%pos%',most_recent = True, remove_empty = True, filter_list = [])
for g in groups:
    print g.name

20160413_Bhedlund_pHILIC_POS_GBS_4OE_Ctrl_Set1
20160413_Bhedlund_pHILIC_POS_G233_BMR_Set1
20160413_Bhedlund_pHILIC_POS_GBS_32OE_inc_Set2
20160413_Bhedlund_pHILIC_POS_JAD2_GBS_32OE_Set1
20160413_Bhedlund_pHILIC_POS_QCmix_Set1
20160413_Bhedlund_pHILIC_POS_Blank_Set1
20160413_Bhedlund_pHILIC_POS_Blank_Set2
20160413_Bhedlund_pHILIC_POS_JKG1_PR_Set1
20160413_Bhedlund_pHILIC_POS_JAD2_GBS_Set1
20160413_Bhedlund_pHILIC_POS_BMR_Beg_Set1
20160413_Bhedlund_pHILIC_POS_JAD2_GBS_4OE_Set1
20160413_Bhedlund_pHILIC_POS_BMR_End_Set1
20160413_Bhedlund_pHILIC_POS_GBS_conc_inc_Set2
20160413_Bhedlund_pHILIC_POS_GBS_Med_Ctrl_Set1
20160413_Bhedlund_pHILIC_POS_GBS_32OE_Ctrl_Set1
20160413_Bhedlund_pHILIC_POS_GBS_Med_inc_Set1
20160413_Bhedlund_pHILIC_POS_JAD2_GBS_32OE_Set2
20160413_Bhedlund_pHILIC_POS_PR_End_Set1
20160413_Bhedlund_pHILIC_POS_GBS_32OE_Ctrl_Set2
20160413_Bhedlund_pHILIC_POS_PR_Beg_Set1


#Select Atlas to use

In [3]:
dp = reload(dp)
atlas = dp.get_metatlas_atlas(name='%master%',do_print = True)


0 20160224_KBL_Master_Atlas_Negative_QE_Hilic 2016-02-25 00:06:46
1 katherine_master_atlas_pos_ALL_20160330 2016-03-31 16:49:48
2 20160219_KBL_Negative_QE_HILIC_Master_Common 2016-02-19 23:22:01
3 20160224_KBL_Master_Atlas_Positive_QE_Hilic 2016-02-25 00:08:48
4 20160219_KBL_Positive_QE_HILIC_Master_Common 2016-02-19 23:21:10
5 katherine_master_atlas_neg_ALL_20160330 2016-04-13 19:30:44


In [4]:
myAtlas = atlas[3]

#Get Data and Save it

In [9]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
dp = reload(dp)
output_filename = '/global/homes/b/bpb/notebooks/MetAtlas_Workflow_20151123/20160425_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'
data = dp.get_data_for_groups_and_atlas(groups,myAtlas,output_filename,use_set1 = True)

0 20 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150324_LPSilva_BHedlund_chloroflexi_POS_rerun/20150320_pHILIC_POS_MSMS_BH46.h5


NameError: global name 'ma_data' is not defined

#Load a pre-existing datafile

In [7]:
my_file = '/global/homes/b/bpb/notebooks/MetAtlas_Workflow_20151123/20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'
project_label = '20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas'
# data = dp.get_data(my_file)

#Make Supplementary Tables

In [23]:
peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')

#Error bar

In [28]:
dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')

#Make Chromatograms

In [9]:
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 12, scale_y=True , output_loc=project_label+'/all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 12, scale_y=True , output_loc=project_label+'/all_files_each_compound')

7
74
nrows =  8
length of ymax is  5620


#Make Identification Figures

In [18]:
dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')

#Export Atlas To Spreadsheet

In [5]:
dp = reload(dp)
df = dp.export_atlas_to_spreadsheet(myAtlas,'/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/save_atlas.csv')
df.head()

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3035: DtypeWarning: Columns (5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,inchi,mono_isotopic_molecular_weight,creation_time,description,formula,functional_sets,last_modified,reference_xrefs,synonyms,unique_id,...,source_database,common_name,pubchem_compound_id,lipidmaps_id,metacyc_id,hmdb_id,img_abc_id,chebi_id,kegg_id,metatlas_inchi
4-methylumbelliferone,InChI=1S/C10H8O3/c1-6-4-10(12)13-9-5-7(11)2-3-...,176.047,1444322232,NaN,C10H8O3,NaN,1460575844,NaN,7-Hydroxy-4-methylcoumarin///Imecromone///beta...,67b0de24bdf64e66a3818ac3212b1747,...,enzo///metacyc///wikidata///hmdb///gnps///chebi,4-Methylumbelliferone///4-methylumbelliferone,5280567,nan,CPD-182,HMDB59622,nan,CHEBI:17224,C03081,InChI=1S/C10H8O3/c1-6-4-10(12)13-9-5-7(11)2-3-...
5-aminopentanoate,"InChI=1S/C5H11NO2/c6-4-2-1-3-5(7)8/h1-4,6H2,(H...",117.079,1444322178,NaN,C5H11NO2,NaN,1460575844,NaN,delta-amino-N-valerate///5-Aminovalerate///&de...,797de57641ec415aa1f7a92827ef54c5,...,chebi///msmls///lipidmaps///metacyc///hmdb,5-aminopentanoate///5-aminopentanoic acid zwit...,138///6992101///138.0,LMFA01100040,5-AMINOPENTANOATE,HMDB03355,nan,CHEBI:356010///CHEBI:15887///CHEBI:86394,C00431///C00431,"InChI=1S/C5H11NO2/c6-4-2-1-3-5(7)8/h1-4,6H2,(H..."
2-<i>trans</i>-abscisate,InChI=1S/C15H20O4/c1-10(7-13(17)18)5-6-15(19)1...,264.136,1444322247,NaN,C15H20O4,NaN,1444322293,NaN,2-trans-abscisic acid anion///abscisic acid///...,75359ff3b38c45bb9e87cf292349eaeb,...,chebi///metacyc,2-trans-abscisate///(+)-abscisate///(R)-2-tran...,7251168///6951345,nan,CPD-693///CPD-7731,nan,nan,CHEBI:37569///CHEBI:62429///CHEBI:62421///CHEB...,C06082///C06082,InChI=1S/C15H20O4/c1-10(7-13(17)18)5-6-15(19)1...
2-<i>trans</i>-abscisate,InChI=1S/C15H20O4/c1-10(7-13(17)18)5-6-15(19)1...,264.136,1444322247,NaN,C15H20O4,NaN,1444322293,NaN,2-trans-abscisic acid anion///abscisic acid///...,75359ff3b38c45bb9e87cf292349eaeb,...,chebi///metacyc,2-trans-abscisate///(+)-abscisate///(R)-2-tran...,7251168///6951345,nan,CPD-693///CPD-7731,nan,nan,CHEBI:37569///CHEBI:62429///CHEBI:62421///CHEB...,C06082///C06082,InChI=1S/C15H20O4/c1-10(7-13(17)18)5-6-15(19)1...
adenine,InChI=1S/C5H5N5/c6-4-3-5(9-1-7-3)10-2-8-4/h1-2...,135.054,1444322234,NaN,C5H5N5,NaN,1460575844,NaN,6-aminopurine///1H-Purin-6-amine///Adeninimine...,e4e7763b15e5490983bb51ba8f44e556,...,chebi///msmls///wikidata///metacyc///hmdb,Adenine///adenine,190.0///190,nan,ADENINE,HMDB00034,nan,CHEBI:16708,C00147,InChI=1S/C5H5N5/c6-4-3-5(9-1-7-3)10-2-8-4/h1-2...
